In [1]:
from cellpose.gui import gui
from pipeline.test import run_gui

/opt/conda/envs/cp_dock/lib/python3.8/site-packages/pyqtgraph/colors/palette.py:1: RuntimeWarning: PyQtGraph supports Qt version >= 5.15, but 5.12.1 detected.
  from ..Qt import QtGui


2024-02-17 19:07:41,263 [INFO] WRITING LOG OUTPUT TO /root/.cellpose/run.log
2024-02-17 19:07:41,265 [INFO] 
cellpose version: 	2.2.3 
platform:       	linux 
python version: 	3.8.18 
torch version:  	1.11.0


: 

In [5]:
run_gui()

2024-02-17 19:05:14,779 [INFO] WRITING LOG OUTPUT TO /root/.cellpose/run.log
2024-02-17 19:05:14,784 [INFO] 
cellpose version: 	2.2.3 
platform:       	linux 
python version: 	3.8.18 
torch version:  	1.11.0


: 